In [8]:
# Import modules
import sys
sys.path.append('..')
from src.data_processing import DataPreprocessor
from src.model_training import ModelTrainer

In [9]:
# Setup
model_name = "distilbert-base-uncased"
output_dir = "../models/sentiment_model"

In [10]:
# Prepare data
print("Preparing dataset...")
preprocessor = DataPreprocessor(model_name, max_length=256)
tokenized_dataset = preprocessor.load_and_prepare_data(
    "imdb",
    subset_size=1000
)

Preparing dataset...
Loading imdb...
Tokenizing the dataset...


In [ ]:
# Cell 4: VERIFY label balance
train_labels = np.array(tokenized_dataset['train']['labels'])
test_labels = np.array(tokenized_dataset['test']['labels'])

print("Training set label distribution:")
print(f"  Negative (0): {np.sum(train_labels == 0)} ({np.mean(train_labels == 0)*100:.1f}%)")
print(f"  Positive (1): {np.sum(train_labels == 1)} ({np.mean(train_labels == 1)*100:.1f}%)")

print("\nTest set label distribution:")
print(f"  Negative (0): {np.sum(test_labels == 0)} ({np.mean(test_labels == 0)*100:.1f}%)")
print(f"  Positive (1): {np.sum(test_labels == 1)} ({np.mean(test_labels == 1)*100:.1f}%)")

In [11]:
# Initialize trainer
trainer_obj = ModelTrainer(model_name, num_labels=2)
training_args = trainer_obj.setup_training(output_dir=output_dir)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


loading model: distilbert-base-uncased


In [12]:
# Train model
trainer = trainer_obj.train(
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    training_args=training_args
)

Starting training...


/Users/xander/Desktop/AI/AI_Projects/llm-fine-tuning/virtual/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.051100,0.000873,1.000000,0.000000,0.000000,0.000000
2,0.000800,0.000354,1.000000,0.000000,0.000000,0.000000
3,0.000400,0.000277,1.000000,0.000000,0.000000,0.000000


/Users/xander/Desktop/AI/AI_Projects/llm-fine-tuning/virtual/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/xander/Desktop/AI/AI_Projects/llm-fine-tuning/virtual/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/xander/Desktop/AI/AI_Projects/llm-fine-tuning/virtual/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this

Model saved to ../models/sentiment_model


In [13]:
# Evaluate on test set
print("\nFinal evaluation on test set:")
results = trainer.evaluate()
for key, value in results.items():
    print(f"{key}: {value:.4f}")


Final evaluation on test set:


/Users/xander/Desktop/AI/AI_Projects/llm-fine-tuning/virtual/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


eval_loss: 0.0009
eval_accuracy: 1.0000
eval_precision: 0.0000
eval_recall: 0.0000
eval_f1: 0.0000
eval_runtime: 16.1389
eval_samples_per_second: 12.3920
eval_steps_per_second: 0.8060
epoch: 3.0000


/Users/xander/Desktop/AI/AI_Projects/llm-fine-tuning/virtual/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/xander/Desktop/AI/AI_Projects/llm-fine-tuning/virtual/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/xander/Desktop/AI/AI_Projects/llm-fine-tuning/virtual/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this

In [14]:
# Save tokenizer needed for inference
preprocessor.tokenizer.save_pretrained(output_dir)
print(f"\nTokenizer saved to {output_dir}")


Tokenizer saved to ../models/sentiment_model
